<a href="https://colab.research.google.com/github/avelo/stuff/blob/master/excel2whpexchange.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import files
uploaded_spreadsheet_file=files.upload()
institution='CREATOR'  #@param {type: "string"}
#
spreadsheet_filename=list(uploaded_spreadsheet_file.keys())[0]
ext = spreadsheet_filename.split('.')[-1]
if not ext in ['xlsx', 'ods']:
    error(f'Only spreadsheets in Excel (.xlsx) or OpenDocument (.ods) format allowed') 
if ext == 'ods':
    !pip install odfpy

ModuleNotFoundError: No module named 'google.colab'

In [5]:
%load_ext google.colab.data_table
import io
import re
import csv
import datetime as dt
import pandas as pd
from google.colab import data_table
from google.colab import files

PRETTY_PRINT = True
#
## Define Precisions
precision = {}
for k in {'STNNBR', 'CASTNO', 'BTLNBR', 'SAMPNO'}:
    precision[k] = 0
for k in {'CTDOXY', 'OXYGEN', 'SILCAT', 'NITRAT', 'NITRIT', 'PHSPHT', 'TCARBN', 'ALKALI', 'CFC113', 'SF6', 'CCL4', 'HELIUM', 'HELIUM_ERR', 'DELHE3', 'DELHE3_ERR'}:
    precision[k] = 2
for k in {'CFC11', 'CFC12', 'TRITUM', 'TRITUM_ERR'}:
    precision[k] = 3
for k in {'LATITUDE', 'LONGITUDE', 'CTDPRS', 'CTDTMP', 'CTDSAL', 'SALNTY', 'PHTS25P0', 'NEON', 'NEON_ERR'}:
    precision[k] = 4


## Read FIRST sheet with data
engine = 'odf' if ext == 'ods' else 'xlrd'
print(f'Processing {spreadsheet_filename}')
df = pd.read_excel(io.BytesIO(uploaded_spreadsheet_file[spreadsheet_filename]), 0, dtype=object, encoding='utf-8', engine=engine)
units = df.iloc[0, :]
df = df.iloc[1:, :]
# Set precisions
for k, v in precision.items():
    try:
        df[k] = df[k].astype(float).apply(lambda x: f'{x:{9 if PRETTY_PRINT else ""}.{v}f}')
    except:
        pass
        #print(f'.. unable to format or unexistent: {k}')
try:
    df['TIME'] = df['TIME'].astype(float).apply(lambda x: f'{x:04.0f}')
except:
    pass
# Remove -999 cols
cols_to_remove = []
cols_to_remove2 = []
for col in df:
    if (df[col].astype(float, errors='ignore') == -999).all():
        cols_to_remove.append(col)
        if f'{col}_FLAG_W' in df:
            cols_to_remove2.append(f'{col}_FLAG_W')
print(f'..following columns removed due to -999: {",".join(cols_to_remove)}')
print(f'..following columns removed due to nonsense without previous ones: {",".join(cols_to_remove2)}')
cols_to_remove.extend(cols_to_remove2)
df = df.drop(columns=cols_to_remove)
units = units.drop(cols_to_remove)
## Read SECOND sheet with metadata
try:
    metadata = pd.read_excel(SPREADSHEET_FILE, 1, dtype=str, encoding='utf-8', engine=engine, header=None, na_filter=False).to_csv(header=False, index=False, encoding='utf8', quoting=csv.QUOTE_NONE, escapechar=',')
except:
    metadata = ''
# Write file
outfile = spreadsheet_filename.split('.')[0] + '_hy1.csv'
print(f'Writting WHP-Exchange BOTTLE file in: {outfile}')
with open(outfile, 'wt', encoding='utf-8', newline='') as fout:
    yyyymmdd = dt.datetime.now().strftime('%Y%m%d')
    fout.write(f'BOTTLE,{yyyymmdd}{institution}\n')
    for line in metadata.splitlines():
        line = re.sub(r',+', ',', line).rstrip(',')
        fout.write(f'# {line}\n')
    fout.write(','.join(df.columns)+'\n')
    fout.write(','.join(list(units.fillna('')))+'\n')
    fout.write(df.to_csv(header=False, index=False, line_terminator='\n'))
    fout.write('END_DATA')
print('Done')
files.download(outfile)


ModuleNotFoundError: No module named 'google.colab'

In [6]:
data_table.DataTable(df,max_columns=100,include_index=False)

NameError: name 'data_table' is not defined